# Задача: создание комплексного отчета по работе приложения

## Цель задачи

Создать **комплексный аналитический отчет**, который дает представление о работе всего приложения как единой системы.  
Отчет должен не просто отображать данные, но и **помогать бизнесу отвечать на ключевые вопросы** о состоянии продукта.

---

## Что должен включать отчет?

### 1. **Ключевые метрики приложения как целого**

- DAU / WAU / MAU
- Sticky Factor (DAU / MAU)
- Уровень вовлеченности (например, CTR для ленты, количество сообщений на пользователя для мессенджера)
- Retention
- Доли пользователей по источникам (organic vs ads)

### 2. **Метрики по модулям приложения**
Отчет должен включать показатели по отдельным частям приложения:
#### Новостная лента:
- DAU ленты
- Просмотры и лайки
- CTR
- Retention пользователей ленты

#### Мессенджер:
- DAU мессенджера
- Количество отправленных / полученных сообщений
- Число активных диалогов
- Retention пользователей мессенджера

### 3. **Динамика метрик**
Покажем, как менялись метрики за последние:
- 7 дней
- 30 дней
- Месяц к месяцу

### 4. **Визуализации**
Добавим графики и диаграммы:
- График DAU и активности по дням
- Динамика CTR
- Сравнение количества сообщений
- Тепловые карты retention
- Доли пользователей по источникам

In [ ]:
import telegram
import matplotlib.pyplot as plt
import seaborn as sns
import io
import pandas as pd
import pandahouse as ph
from airflow.decorators import dag, task
from datetime import datetime, timedelta

# Настройки бота
my_token = '************************'
bot = telegram.Bot(token=my_token)

# Подключение к ClickHouse
connection = {
    'host': '*****',
    'password': '*****',
    'user': '*****',
    'database': '*****'
}

# Параметры DAG
default_args = {
    'owner': 'm.kruzhalin',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2025, 6, 28)
}

schedule_interval = '0 11 * * *'

@dag(default_args=default_args, schedule_interval=schedule_interval, catchup=False)
def maximkruzhalin_dag_tg_rep11():

    @task()
    def extract_age_groups():
        """Данные по возрастным группам в мессенджере"""
        q = """
        SELECT 
            CASE 
                WHEN age < 11 THEN 'Дети (0-11)'
                WHEN age BETWEEN 11 AND 17 THEN 'Подростки (11-18)'
                WHEN age BETWEEN 18 AND 24 THEN 'Молодежь (18-25)'
                WHEN age BETWEEN 25 AND 39 THEN 'Взрослые (25-40)'
                WHEN age BETWEEN 40 AND 54 THEN 'Зрелые (40-55)'
                ELSE 'Пожилые (55+)'
            END AS age_group,
            COUNT(*) AS messages_count,
            COUNT(DISTINCT user_id) AS unique_users
        FROM simulator_20250520.message_actions
        WHERE toDate(time) = today() - 1
        GROUP BY age_group
        ORDER BY messages_count DESC
        """
        return ph.read_clickhouse(q, connection=connection)

    @task()
    def extract_common_users():
        """Данные о пересечении пользователей"""
        q = """
        SELECT
            COUNT(DISTINCT m.user_id) AS messenger_users,
            COUNT(DISTINCT f.user_id) AS feed_users,
            COUNT(DISTINCT CASE WHEN m.user_id IS NOT NULL AND f.user_id IS NOT NULL 
                           THEN m.user_id END) AS common_users,
            COUNT(DISTINCT u.user_id) AS total_users,
            common_users / total_users AS overlap_percent
        FROM 
            (SELECT DISTINCT user_id FROM simulator_20250520.message_actions
             WHERE toDate(time) = today() - 1) m
        FULL OUTER JOIN
            (SELECT DISTINCT user_id FROM simulator_20250520.feed_actions
             WHERE toDate(time) = today() - 1) f
        USING (user_id)
        CROSS JOIN
            (SELECT DISTINCT user_id FROM (
                SELECT user_id FROM simulator_20250520.message_actions WHERE toDate(time) = today() - 1
                UNION DISTINCT
                SELECT user_id FROM simulator_20250520.feed_actions WHERE toDate(time) = today() - 1
            )) u
        """
        return ph.read_clickhouse(q, connection=connection)

    @task()
    def send_age_report(age_df):
    """Отправка графика по возрастам"""

        plt.figure(figsize=(12, 6))
        age_df = age_df.sort_values('messages_count', ascending=False)

        ax = sns.barplot(data=age_df, x='age_group', y='messages_count', palette='viridis')
        plt.title('Сообщения по возрастным группам (вчера)', pad=20)
        plt.xlabel('')
        plt.ylabel('Кол-во сообщений')
        plt.xticks(rotation=45)

        for i, p in enumerate(ax.patches):
            ax.annotate(f"{int(p.get_height())}\n({age_df.iloc[i]['unique_users']} юзеров)", 
                       (p.get_x() + p.get_width() / 2., p.get_height()),
                       ha='center', va='center', xytext=(0, 10), textcoords='offset points')

        plot_object = io.BytesIO()
        plt.savefig(plot_object, format='png', bbox_inches='tight', dpi=100)
        plot_object.seek(0)
        plt.close()

        bot.sendPhoto(
            chat_id=*****,
            photo=plot_object,
            caption="📊 Распределение сообщений по возрастам"
            )
        
    @task()
    def send_common_users_report(common_df):
        """Отправка отчета о пересечении"""
        message = f"""
📊 Пересечение пользователей за вчера:

• Юзеров мессенджера: {common_df['messenger_users'].iloc[0]}
• Юзеров ленты: {common_df['feed_users'].iloc[0]}
• Общих юзеров: {common_df['common_users'].iloc[0]}
• Всего уникальных: {common_df['total_users'].iloc[0]}
• Пересечение: {common_df['overlap_percent'].iloc[0]*100:.1f}%
        """
        bot.sendMessage(
            chat_id=*****,
            text=message
        )

    # Получение данных
    age_data = extract_age_groups()
    common_data = extract_common_users()

    # Отправка отчетов
    send_age_report(age_data)
    send_common_users_report(common_data)

maximkruzhalin_dag_tg_rep2 = maximkruzhalin_dag_tg_rep2()

![tg_bot2](tg_bot_2.png)